Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [1]:
!pip install corus

In [2]:
!pip install stop_words

In [3]:
!pip install annoy

In [4]:
!pip install pymorphy2

In [55]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm

In [56]:
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)

NewsRecord(
    timestamp=datetime.datetime(2008, 11, 21, 15, 19, 14),
    url='https://www.gazeta.ru/news/business/2008/11/21/n_1298950.shtml',
    edition=None,
    topics='Бизнес',
    authors=None,
    title='Госдума сокращает срок действия ставки экспортных пошлин на нефть',
    text='Госдума приняла сегодня в первом чтении и сразу в целом поправки в закон «О таможенном тарифе», сокращающие срок действия ставки экспортных пошлин на нефть с 2-х до 1-го месяца.nnДля установления средних цен на нефть марки Urals и расчета экспортных пошлин правительство России в течение двух месяцев проводит мониторинг на международных рынках нефтяного сырья (средиземноморском и роттердамском), экспортные пошлины на нефть устанавливаются также раз в два месяца.nnСокращение на месяц периода мониторинга (с 15-го числа каждого календарного месяца по 14-е число следующего месяца) и соответственно срока действия ставок экспортных пошлин «позволит более оперативно реагировать на изменения экономической сит

In [57]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [58]:
# Предварительная подготовка данных
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for line in tqdm(records):
    spls = preprocess_txt(line.text)
    sentences.append(spls)
    c += 1
    if c > 100000:
        break

0it [00:00, ?it/s]

In [59]:
sentences = [i for i in sentences if len(i) > 2]

In [60]:
sentences[0]

['японский',
 'учёный',
 'физический',
 'центр',
 'riken',
 'заявлять',
 'удаться',
 'синтезировать',
 'атом',
 '113го',
 'элемент',
 'таблица',
 'менделеев',
 'элемент',
 'впервые',
 'получить',
 '2003',
 'российский',
 'американский',
 'физик',
 'эксперимент',
 'синтез',
 '115го',
 'элемент',
 'открытие',
 'признать',
 'международный',
 'союз',
 'теоретический',
 'прикладной',
 'химия',
 'сообщаетnриа',
 '«новости»nnть',
 'природа',
 'существовать',
 'элемент',
 'атомный',
 'номер',
 'число',
 'протон',
 'ядро',
 'атом',
 'большой',
 '92',
 'тяжёлый',
 'уран',
 'тяжёлый',
 'элемент',
 'например',
 'плутоний',
 'нарабатываться',
 'атомный',
 'реактор',
 'элемент',
 'тяжёлый',
 '100го',
 'фермий',
 'получать',
 'ускоритель',
 'путём',
 'бомбардировка',
 'мишень',
 'тяжёлый',
 'ион',
 'слияние',
 'ядро',
 'мишень',
 '«снаряда»',
 'возникать',
 'ядро',
 'новый',
 'элементаnnгруппа',
 'учёный',
 'руководство',
 'косука',
 'морить',
 'статья',
 'опубликовать',
 'journal',
 'of',
 'physical

In [61]:
# Модель W2V

modelW2V = Word2Vec(sentences = sentences, vector_size = 300, window = 5, min_count = 1)

In [62]:
# Модель FT

modelFT = FastText(sentences = sentences, vector_size = 300, min_count = 1, window = 5, workers = 8)

In [63]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm(records):
        n_w2v = 0
        n_ft = 0
        index_map[counter] = line.text
        question = preprocess_txt(line.title)
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [64]:
def get_response(question, index, model, index_map, most_similar = 3):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    
    answers = index.get_nns_by_vector(vector, most_similar, )
    return [index_map[i] for i in answers]

In [65]:
TEXT = "какой город самы красивый"

In [66]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Жители немецкого города Гамбурга на референдуме проголосовали против выдвижения кандидатуры города на проведение летних Олимпийских игр 2024 года. Об этом сообщает агентствоnReutersnсо ссылкой на мэра города.nПочти 52% респондентов высказались против.nГамбург уже подал официальную заявку на проведение Олимпиады. Администрация города планировала потратить на проведение соревнований около $12,6 млрд.nНапомним, в сентябре Международный олимпийский комитет (МОК) объявил окончательный список кандидатов на проведение летних Олимпийских игр 2024 года: в него вошли Гамбург (Германия), Рим (Италия), Лос-Анджелес (США), Париж (Франция) и Будапешт (Венгрия). В апреле — мае 2016 года МОК окончательно утвердит число городов-кандидатов, претендующих на организацию соревнований.nОзнакомиться с другими материалами, новостями и статистикой можно наnстраницеnлетних видов спорта, а также в группах отдела спорта в социальных сетяхnFacebooknиn«ВКонтакте»n.',
 'Города — организаторы чемпионата мира по фут

In [67]:
get_response(TEXT, ft_index, modelFT, index_map)

['Заместитель мэра Москвы по градостроительной политике и строительству Марат Хуснуллин выразил мнение, что в городе достаточно гостиниц для проведения чемпионата мира по футболу 2018 года в России.n«Мы вопросом занимаемся, но это не от нас зависит. Правила диктует рынок. Гостиницы — это сегмент недвижимости с очень длительным сроком окупаемости. Но я могу сказать, что в Москве сегодня гостиниц для проведения чемпионата мира достаточно. Всех болельщиков, кто приедет на чемпионат мира, мы разместим без проблем.nnПри этом надо понимать, что программа правительства Москвы по развитию гостиничной сети в столице связана не только с чемпионатом мира по футболу, но и с необходимостью развития туристической инфраструктуры города в целом», — приводит слова ХуснуллинаnРИА «Новости»n.nОзнакомиться с другими новостями, материалами и статистикой вы можете наnстраницеnчемпионата мира по футболу — 2018.',
 'Системы видеонаблюдения Нижнего Новгорода подключат к программному комплексу «Безопасный город

In [68]:
get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['Жители немецкого города Гамбурга на референдуме проголосовали против выдвижения кандидатуры города на проведение летних Олимпийских игр 2024 года. Об этом сообщает агентствоnReutersnсо ссылкой на мэра города.nПочти 52% респондентов высказались против.nГамбург уже подал официальную заявку на проведение Олимпиады. Администрация города планировала потратить на проведение соревнований около $12,6 млрд.nНапомним, в сентябре Международный олимпийский комитет (МОК) объявил окончательный список кандидатов на проведение летних Олимпийских игр 2024 года: в него вошли Гамбург (Германия), Рим (Италия), Лос-Анджелес (США), Париж (Франция) и Будапешт (Венгрия). В апреле — мае 2016 года МОК окончательно утвердит число городов-кандидатов, претендующих на организацию соревнований.nОзнакомиться с другими материалами, новостями и статистикой можно наnстраницеnлетних видов спорта, а также в группах отдела спорта в социальных сетяхnFacebooknиn«ВКонтакте»n.',
 'Города — организаторы чемпионата мира по фут

In [69]:
get_response(TEXT, ft_index, modelFT, index_map, 5)

['Заместитель мэра Москвы по градостроительной политике и строительству Марат Хуснуллин выразил мнение, что в городе достаточно гостиниц для проведения чемпионата мира по футболу 2018 года в России.n«Мы вопросом занимаемся, но это не от нас зависит. Правила диктует рынок. Гостиницы — это сегмент недвижимости с очень длительным сроком окупаемости. Но я могу сказать, что в Москве сегодня гостиниц для проведения чемпионата мира достаточно. Всех болельщиков, кто приедет на чемпионат мира, мы разместим без проблем.nnПри этом надо понимать, что программа правительства Москвы по развитию гостиничной сети в столице связана не только с чемпионатом мира по футболу, но и с необходимостью развития туристической инфраструктуры города в целом», — приводит слова ХуснуллинаnРИА «Новости»n.nОзнакомиться с другими новостями, материалами и статистикой вы можете наnстраницеnчемпионата мира по футболу — 2018.',
 'Системы видеонаблюдения Нижнего Новгорода подключат к программному комплексу «Безопасный город

In [70]:
TEXT = "Китай в миниатюре"

get_response(TEXT, w2v_index, modelW2V, index_map, 5)

['В нападающем дортмундской «Боруссии» Пьер-Эмерике Обамеянге заинтересован «Тяньцзинь Цюаньцзянь», передаетnRTn.nКитайцы уже предложили за габонца €80 млн, сообщает The Sun.nСам футболист может рассчитывать на зарплату размером в €30 млн за сезон.',
 'Китайский клуб «Шанхай Шэньхуа» хочет приобрести нападающего «Галатасарая» Лукаса Подольски, утверждаетnBildn.nПо информации источника, в Китае немец будет получать по €9 млн за сезон. Отмечается, что турецкий коллектив планирует выручить за форварда около €20 млн.nВ текущем сезоне Подольски провел в Суперлиге 17 матчей, забив восемь голов.nДругие новости, материалы и статистику можно посмотреть наnстраницеnмирового футбола, а также в группах отдела спорта в социальных сетяхnFacebooknиn«ВКонтакте»n.',
 'Российская спортсменка Марта Мартьянова заняла второе место в индивидуальных соревнованиях рапиристок на этапе Гран-при по фехтованию в Лонг-Бич, уступив в решающем поединке американке Ли Кифер.nТретье место заняли россиянка Инна Дериглаз

In [71]:
get_response(TEXT, ft_index, modelFT, index_map, 5)

['Во время проведения полумарафона в китайском городе Шэньчжэнь умер 33-летний мужчина, сообщает агентствоn«Синьхуа»n.nПо заявлению организаторов, бегун по фамилии Яо неожиданно упал в 400 м от финиша на соревновании. Отмечается, что мужчине была сразу оказана вся необходимая медицинская помощь, так как медики были рядом с местом проведения забега.nБегун не приходил в сознание в пути до больницы. Тем не менее врачи в медицинском учреждении также не смогли спасти ему жизнь.nnЯо не первый раз бежал марафон. В прошлом месяце он пробежал полумарафон за 2 часа 10 минут. Именно поэтому многие были удивлены произошедшим.nОзнакомиться с другими материалами, новостями и статистикой можно наnстраницеnлетних видов спорта, а также в группах отдела спорта в социальных сетяхnFacebooknиn«ВКонтакте»n.',
 'Действующий чемпион MotoGP Марк Маркес стал победителем этапа серии в США.nВторым стал семикратный чемпион Валентино Росси, третьим — Дани Педроса.nЛидер сезона Маверик Виньялес снялся с дистанции в 

Если с лучшим городом в мире обе модельки худо-бедно справились, то об Китай сломали зубы тоже обе, некоторые статьи никакого отношения к Китаю не имеют